In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings

warnings.filterwarnings(action='ignore')
%matplotlib inline

In [ ]:
paths = '/content/drive/MyDrive/DataAnalysis/M5 Forecasting - Accuracy/input/'

calendar = pd.read_csv(paths + 'calendar.csv')
ste = pd.read_csv(paths + 'sales_train_evaluation.csv')
stv = pd.read_csv(paths + 'sales_train_validation.csv')
ss = pd.read_csv(paths + 'sample_submission.csv')
sp = pd.read_csv(paths + 'sell_prices.csv')
sp['cat_id'] = sp['item_id'].apply(lambda x : x.split('_')[0])

# 데이터 확인

### File 1: “calendar.csv” 
Contains information about the dates the products are sold.
- date: The date in a “y-m-d” format.
- wm_yr_wk: The id of the week the date belongs to.
- weekday: The type of the day (Saturday, Sunday, …, Friday).
- wday: The id of the weekday, starting from Saturday.
- month: The month of the date.
- year: The year of the date.
- event_name_1: If the date includes an event, the name of this event.
- event_type_1: If the date includes an event, the type of this event.
- event_name_2: If the date includes a second event, the name of this event.
- event_type_2: If the date includes a second event, the type of this event.
- snap_CA, snap_TX, and snap_WI: A binary variable (0 or 1) indicating whether the stores of CA, TX or WI allow SNAP  purchases on the examined date. 1 indicates that SNAP purchases are allowed.

### File 2: “sell_prices.csv”
Contains information about the price of the products sold per store and date.
- store_id: The id of the store where the product is sold. 
- item_id: The id of the product.
- wm_yr_wk: The id of the week.
- sell_price: The price of the product for the given week/store. The price is provided per week (average across seven days). If not available, this means that the product was not sold during the examined week. Note that although prices are constant at weekly basis, they may change through time (both training and test set).  

### File 3: “sales_train.csv” 
Contains the historical daily unit sales data per product and store.
- item_id: The id of the product.
- dept_id: The id of the department the product belongs to.
- cat_id: The id of the category the product belongs to.
- store_id: The id of the store where the product is sold.
- state_id: The State where the store is located.
- d_1, d_2, …, d_i, … d_1941: The number of units sold at day i, starting from 2011-01-29. 


In [ ]:
calendar.head(20)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,NaN,1,1,1
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,NaN,1,0,0
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,NaN,1,1,1
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,1,1,1
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,NaN,1,1,0


In [ ]:
ste.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,0,3,1,4,1,0,0,3,4,4,0,0,1,0,1,1,7,7,3,6,3,3,7,12,4,2,7,5,12,5,...,0,0,2,0,0,0,0,0,1,0,0,1,0,0,0,2,2,0,0,0,2,0,0,1,1,1,0,0,1,0,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,2,...,0,0,2,1,0,2,0,1,0,0,1,0,0,0,1,1,0,2,1,1,0,0,1,0,0,1,0,3,3,1,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,3,0,0,1,0,0,1,0,3,1,3,1,3,0,1,2,1,0,2,1,1,2,0,2,1,1,0,2,1,1,1,1,4,6,0,1,1,1,0
30489,FOODS_3_827_WI_3_evaluation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,2,0,1,3,2,1,1,0,0,0,1,2,0,5,4,0,2,2,5,1


In [ ]:
ste[ste['store_id'] == 'CA_1']

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,3,0,1,1,1,3,0,1,1,0,0,0,2,0,3,5,0,0,1,1,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,2,2,1,2,1,1,1,0,1,1,1,0,0,1,1,0,2,1,0,0,0,0,2,1,3,0,0,1,0,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,0,1,0,5,4,1,0,1,3,7,2,0,0,1,2,4,1,6,4,0,0,0,2,2,4,2,1,1,1,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,2,1,1,0,1,1,2,2,2,4,1,0,2,3,1,0,3,2,3,1,1,3,2,3,2,2,2,2,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,FOODS_3_823_CA_1_evaluation,FOODS_3_823,FOODS_3,FOODS,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,2,4,0,2,1,0,2,0,4,1,1,0,1,1,1,1,4,0,3,1,0,0,2,0,1,0,2,2,2,3,0,3,1,0,2,4,3,4,2
3045,FOODS_3_824_CA_1_evaluation,FOODS_3_824,FOODS_3,FOODS,CA_1,CA,1,0,5,0,0,3,0,0,2,1,1,1,0,1,4,3,0,0,0,0,4,3,1,0,3,0,0,0,0,2,2,0,0,2,...,0,0,1,0,0,0,1,0,2,2,1,0,1,0,3,0,0,2,0,3,2,2,0,0,2,0,5,3,0,0,0,0,0,1,2,0,3,0,0,0
3046,FOODS_3_825_CA_1_evaluation,FOODS_3_825,FOODS_3,FOODS,CA_1,CA,0,0,0,0,0,0,0,1,2,1,2,0,4,1,10,19,2,1,1,1,2,3,2,0,1,0,0,0,0,1,0,0,0,0,...,1,1,1,2,0,0,1,1,0,1,3,2,2,0,0,1,2,0,3,1,4,0,0,1,1,3,2,1,2,1,0,0,0,2,0,2,0,3,0,1
3047,FOODS_3_826_CA_1_evaluation,FOODS_3_826,FOODS_3,FOODS,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,4,0,2,0,0,3,2,0,0,0,0,1,1,0,0,0,4,0,0,2,0,0,1,3,1,0,0,1,2,1,3,3,0,1,2,1


In [ ]:
sp.head(20)

,store_id,item_id,wm_yr_wk,sell_price,cat_id
0,CA_1,HOBBIES_1_001,11325,9.58,HOBBIES
1,CA_1,HOBBIES_1_001,11326,9.58,HOBBIES
2,CA_1,HOBBIES_1_001,11327,8.26,HOBBIES
3,CA_1,HOBBIES_1_001,11328,8.26,HOBBIES
4,CA_1,HOBBIES_1_001,11329,8.26,HOBBIES
5,CA_1,HOBBIES_1_001,11330,8.26,HOBBIES
6,CA_1,HOBBIES_1_001,11331,8.26,HOBBIES
7,CA_1,HOBBIES_1_001,11332,8.26,HOBBIES
8,CA_1,HOBBIES_1_001,11333,8.26,HOBBIES
9,CA_1,HOBBIES_1_001,11334,8.26,HOBBIES


In [ ]:
ss.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# 데이터 전처리 계획

1. CA, TX, WI 별로 모델을 만들 예정이다. -> 데이터를 3개로 나눕시다.
2. sp 데이터를 활용하여 각 물건 마다의 가격을 나타내는 feature을 만들 예정이다.
3. calendar 데이터를 활용하여 요일 feature을 만들 예정이다.
4. calendar 데이터를 활용하여 event 관련 feature을 만들 예정이다.
5. calendar 데이터를 활용하여 snap 관련 feature을 만들 예정이다.
6. 만들어진 가격 feature를 rolling 하여 mean, std를 7일, 14일, 21일, 28일 기준의 feature를 만들 예정이다.
7. ste 데이터의 target을 rolling 하여 mean, std를 7일, 14일, 21일, 28일 기준의 feature를 만들 예정이다. -> 재귀적 방법을 사용할 경우에는 해당 피쳐를 사용가능

In [ ]:
def get_models_and_data(store, cat):

  names = store.split('_')[0]

  data = ste[(ste['store_id'] == store) & (ste['cat_id'] == cat)]

  pre_ste = data.iloc[:, 6:].T
  pre_ste.columns = data['id'].values.tolist()
  pre_ste = pre_ste.reset_index(drop=True)
  pre_ste = pre_ste.mean(axis = 1)
  pre_ste.columns = 'target'
  pre_ste = pd.concat([calendar, pre_ste], axis = 1)
  pre_ste = pre_ste.rename(columns = {0 : 'target'})
  pre_ste = pre_ste.iloc[-28 * 6:, : ]
  pre_ste = pre_ste.reset_index(drop=True)
  pre_ste['sell_price'] = sp[(sp['store_id'] == store) & (sp['cat_id'] == cat) & (sp['wm_yr_wk'] >= 11549)].mean()['sell_price']

  pre_ste['sales_mean_7'] = pre_ste['target'].shift(1).rolling(window=7).mean()
  pre_ste['sales_mean_14'] = pre_ste['target'].shift(1).rolling(window=14).mean()
  pre_ste['sales_mean_21'] = pre_ste['target'].shift(1).rolling(window=21).mean()
  pre_ste['sales_mean_28'] = pre_ste['target'].shift(1).rolling(window=28).mean()

  pre_ste['sales_std_7'] = pre_ste['target'].shift(1).rolling(window=7).std()
  pre_ste['sales_std_14'] = pre_ste['target'].shift(1).rolling(window=14).std()
  pre_ste['sales_std_21'] = pre_ste['target'].shift(1).rolling(window=21).std()
  pre_ste['sales_std_28'] = pre_ste['target'].shift(1).rolling(window=28).std()

  cols = ['date', 'd','wday', 'snap_' + names, 'sell_price', 
          'sales_mean_7', 'sales_mean_14', 'sales_mean_21', 'sales_mean_28',
          'sales_std_7', 'sales_std_14', 'sales_std_21', 'sales_std_28',
          'target']

  pre_ste = pre_ste[cols]

  paths = '/content/drive/MyDrive/DataAnalysis/M5 Forecasting - Accuracy/data/'
  data_name = store +'_'+ cat + '_' + 'data' +'.csv'
  
  pre_ste.to_csv(paths + data_name, index = False)

  train = pre_ste.iloc[28 : -28 , :].drop(labels = ['date', 'd', 'target'], axis = 1)
  target = pre_ste.iloc[28 : -28 , :]['target']

  import lightgbm as LGB

  lgb_params = {      'random_state' : 22, 
                      # 'boosting_type': 'gbdt',
                      'objective': 'tweedie',
                      # 'tweedie_variance_power': 1.1,
                      # 'metric': 'rmse',
                      # 'subsample': 0.5,
                      # 'subsample_freq': 1,
                      # 'learning_rate': 0.03,
                      # 'num_leaves': 2**11-1,
                      # 'min_data_in_leaf': 2**12-1,
                      # 'feature_fraction': 0.5,
                      # 'max_bin': 100,
                      # 'n_estimators': 1400,
                      # 'boost_from_average': False,
                      # 'verbose': -1,
                  } 

  lgbm = LGB.LGBMRegressor(**lgb_params)
  lgbm.fit(train, target)

  paths = '/content/drive/MyDrive/DataAnalysis/M5 Forecasting - Accuracy/model/'

  model_name = store +'_'+ cat + '_' + 'model' +'.pkl'

  from sklearn.externals import joblib
  joblib.dump(lgbm, paths + model_name)
  print(data_name)
  print(model_name)

In [ ]:
store_li = sp['store_id'].unique().tolist()
cat_li = sp['cat_id'].unique().tolist()
for store in store_li:
  for cat in cat_li:
    get_models_and_data(store, cat)

CA_1_HOBBIES_data.csv
CA_1_HOBBIES_model.pkl
CA_1_HOUSEHOLD_data.csv
CA_1_HOUSEHOLD_model.pkl
CA_1_FOODS_data.csv
CA_1_FOODS_model.pkl
CA_2_HOBBIES_data.csv
CA_2_HOBBIES_model.pkl
CA_2_HOUSEHOLD_data.csv
CA_2_HOUSEHOLD_model.pkl
CA_2_FOODS_data.csv
CA_2_FOODS_model.pkl
CA_3_HOBBIES_data.csv
CA_3_HOBBIES_model.pkl
CA_3_HOUSEHOLD_data.csv
CA_3_HOUSEHOLD_model.pkl
CA_3_FOODS_data.csv
CA_3_FOODS_model.pkl
CA_4_HOBBIES_data.csv
CA_4_HOBBIES_model.pkl
CA_4_HOUSEHOLD_data.csv
CA_4_HOUSEHOLD_model.pkl
CA_4_FOODS_data.csv
CA_4_FOODS_model.pkl
TX_1_HOBBIES_data.csv
TX_1_HOBBIES_model.pkl
TX_1_HOUSEHOLD_data.csv
TX_1_HOUSEHOLD_model.pkl
TX_1_FOODS_data.csv
TX_1_FOODS_model.pkl
TX_2_HOBBIES_data.csv
TX_2_HOBBIES_model.pkl
TX_2_HOUSEHOLD_data.csv
TX_2_HOUSEHOLD_model.pkl
TX_2_FOODS_data.csv
TX_2_FOODS_model.pkl
TX_3_HOBBIES_data.csv
TX_3_HOBBIES_model.pkl
TX_3_HOUSEHOLD_data.csv
TX_3_HOUSEHOLD_model.pkl
TX_3_FOODS_data.csv
TX_3_FOODS_model.pkl
WI_1_HOBBIES_data.csv
WI_1_HOBBIES_model.pkl
WI_1_HOUSE